<a href="https://colab.research.google.com/github/AdamDunn22/FinalProject/blob/Mohamed/Diabetes_AWS_Connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [5]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-22 04:09:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-11-22 04:09:37 (11.4 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [26]:
from pyspark import SparkFiles
# Load in three different CSV from S3 into a DataFrame
url1_patient_info = "https://data1-patient-info.s3.us-east-2.amazonaws.com/data1_patient_info.csv"
spark.sparkContext.addFile(url1_patient_info)

url2_patient_tests = "https://data2-patient-tests.s3.us-east-2.amazonaws.com/data2_patient_tests.csv"
spark.sparkContext.addFile(url2_patient_tests)

url3_patient_results = "https://data3-patient-results.s3.us-east-2.amazonaws.com/data3_patient_results.csv"
spark.sparkContext.addFile(url3_patient_results)

df_patient_info = spark.read.option('header', 'true').csv(SparkFiles.get("data1_patient_info.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df_patient_tests = spark.read.option('header', 'true').csv(SparkFiles.get("data2_patient_tests.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df_patient_results = spark.read.option('header', 'true').csv(SparkFiles.get("data3_patient_results.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")



In [27]:
df_patient_info.show(10)

+--------+---+------+------+------+
|personid|age| sexid|height|weight|
+--------+---+------+------+------+
|       0| 43|female| 166.4|  53.5|
|       1| 34|  male| 169.0|  57.0|
|       2| 32|female| 157.0|  51.0|
|       3| 59|  male| 165.0|  63.0|
|       4| 30|female| 163.5|  48.5|
|       5| 41|female| 157.0|  57.9|
|       6| 51|female| 170.0|  61.8|
|       7| 36|  male| 166.0|  66.7|
|       8| 31|  male| 167.0|  61.5|
|       9| 31|  male| 161.5|  58.5|
+--------+---+------+------+------+
only showing top 10 rows



In [28]:
df_patient_tests.show(10)

+--------+----+---+---+----+-----------+------------+-----------+-----------+----+-----------+----+----+--------------+
|personID| BMI|SBP|DBP| FPG|Cholesterol|Triglyceride|        HDL|        LDL| ALT|        AST| BUN| CCR|FPG_finalvisit|
+--------+----+---+---+----+-----------+------------+-----------+-----------+----+-----------+----+----+--------------+
|       0|19.3| 96| 57|4.99|       5.13|        0.78|1.372496408|2.763126986|10.0|23.99254235|3.08|50.3|          4.97|
|       1|20.0|124| 69|3.51|       4.61|        1.75|       1.09|       3.13|29.1|23.99254235|6.13|83.7|           5.5|
|       2|20.7| 98| 68|4.25|       4.73|        0.47|1.372496408|2.763126986| 6.9|       19.5|4.45|42.8|           4.9|
|       3|23.1|136| 73| 5.7|        4.5|        0.75|       1.53|        2.8|15.0|23.99254235|5.76|62.6|           5.5|
|       4|18.1|107| 76|4.42|       3.48|        0.52|       1.55|       1.43|12.0|       19.9|2.51|48.9|          4.82|
|       5|23.5|119| 83|4.53|        3.7|

In [29]:
df_patient_results.show(10)

+--------+--------+------------+-------------+--------------------+
|personID|diabetes|    smokerID|    drinkerID|           famhistID|
+--------+--------+------------+-------------+--------------------+
|       0|      no|never smoker|never drinker|family history of...|
|       1|      no|     no info|      no info|no family history...|
|       2|      no|     no info|      no info|no family history...|
|       3|      no|never smoker|never drinker|no family history...|
|       4|      no|     no info|      no info|no family history...|
|       5|      no|     no info|      no info|no family history...|
|       6|      no|     no info|      no info|no family history...|
|       7|      no|     no info|      no info|no family history...|
|       8|      no|     no info|      no info|no family history...|
|       9|      no|never smoker|never drinker|no family history...|
+--------+--------+------------+-------------+--------------------+
only showing top 10 rows



## Drop duplicates and incomplete rows

In [16]:
print(df_patient_info.count())
df_patient_info = df_patient_info.dropna()
print(df_patient_info.count())
df_patient_info = df_patient_info.dropDuplicates()
print(df_patient_info.count())

202393
202393
202393


In [31]:
print(df_patient_tests.count())
df_patient_tests = df_patient_tests.dropna()
print(df_patient_tests.count())
df_patient_tests = df_patient_tests.dropDuplicates()
print(df_patient_tests.count())

202393
202393
202393


In [30]:
print(df_patient_results.count())
df_patient_results = df_patient_results.dropna()
print(df_patient_results.count())
df_patient_results = df_patient_results.dropDuplicates()
print(df_patient_results.count())

202393
202393
202393


## Write DataFrame to RDS

In [32]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://diabetes-database.cvg5rnfkuhyi.us-east-2.rds.amazonaws.com:5432/Diabetes_DataBase"
config = {"user":"postgres",
          "password": "123456789",
          "driver":"org.postgresql.Driver"}

In [24]:
# Write DataFrame to table

df_patient_info.write.jdbc(url=jdbc_url, table='patient_info', mode=mode, properties=config)

In [33]:
# Write DataFrame to table

df_patient_tests.write.jdbc(url=jdbc_url, table='patient_tests', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

df_patient_results.write.jdbc(url=jdbc_url, table='patient_results', mode=mode, properties=config)